In [ ]:
# ==========================================
#  BBO-Rietveld v4 Configuration
#  Y2O3 専用設定
# ==========================================
import os

class Config:
    # --------------------------------------
    # 1. 基本設定 (Basic Settings)
    # --------------------------------------
    STUDY_NAME = 'Y2O3_v4'           # プロジェクト名（フォルダ名になります）
    RANDOM_SEED = 1024               # 再現性のためのシード値
    
    # 試行回数 (推奨: 簡易解析=50, 本番=100~200)
    N_TRIALS = 100
    
    # 並列計算数 (-1:全コア使用, 1:通常, 2~4:推奨)
    # ※ Dockerのメモリ制限に注意してください
    N_JOBS = 1

    # --------------------------------------
    # 2. ファイル設定 (File Paths)
    # --------------------------------------
    DATA_DIR = 'data/Y2O3'                  # データがあるフォルダ
    DATA_FILENAME = 'Y2O3.csv'              # 回折データ
    PRM_FILENAME = 'INST_XRY.PRM'           # 装置パラメータ
    
    # 結晶構造ファイル（複数相ある場合はリストに列挙）
    # Y2O3は単相なので1ファイルのみ
    CIF_FILENAMES = ['Y2O3.cif']

    # --------------------------------------
    # 3. 自動/手動 制御設定 (Manual Overrides)
    # 【使い方】
    #  None  : Optunaに任せる (自動探索) [推奨]
    #  値指定 : その値で固定する (手動設定)
    #  True/False : 精密化フラグを強制的にON/OFFする
    # --------------------------------------

    # [A] 解析範囲 (度)
    # Y2O3は一般的に10-90度程度。自動探索させる場合はNone
    MANUAL_LIMITS = None 

    # [B] バックグラウンド設定
    # 関数タイプ (例: 'chebyschev', 'cosine' / None=自動)
    MANUAL_BG_FUNC = None
    # 多項式の次数 (例: 12 / None=自動[3~15])
    MANUAL_BG_DEGREE = None 

    # [C] パラメータ精密化の強制 (ON/OFF)
    # ゼロ点補正 (Zero): 装置由来のズレ。基本は True または None
    FORCE_REFINE_ZERO = None
    
    # 試料変位 (Sample Displacement): 高さズレなど。
    FORCE_REFINE_SAMPLE = None

    # ピーク形状 (U, V, W): 半値幅パラメータ。
    # ※ 計算が発散しやすい場合は False (固定) にすると安定します。
    FORCE_REFINE_PEAK = None 

    # --------------------------------------
    # 4. 制約・判定基準 (Constraints)
    # --------------------------------------
    # 解析失敗とみなすRwpの閾値 (Y2O3は品質が良いデータが多いので80%で十分)
    MAX_RWP_THRESHOLD = 80.0
    
    # 失敗時のペナルティスコア (変更不要)
    ERROR_PENALTY = 1e9

    # 出力先パス (自動生成)
    WORK_DIR = os.path.join('work', STUDY_NAME)

# ディレクトリ初期化
! rm -f {Config.WORK_DIR}/{Config.STUDY_NAME}*
! mkdir -p {Config.WORK_DIR}
print(f"設定完了: 結果は '{Config.WORK_DIR}' に保存されます。")

In [ ]:
import sys
import optuna

# クラス定義: 1回の解析プロジェクトを管理
class ProjectBBO:
    def __init__(self, trial_number):
        # 内部インポートで並列化エラーを回避
        import GSASIIscriptable as G2sc
        
        # プロジェクト作成
        gpx_path = os.path.join(Config.WORK_DIR, f'{Config.STUDY_NAME}_seed{Config.RANDOM_SEED}_trial_{trial_number}.gpx')
        self.gpx = G2sc.G2Project(newgpx=gpx_path)

        # データの読み込み
        self.hist1 = self.gpx.add_powder_histogram(
            os.path.join(Config.DATA_DIR, Config.DATA_FILENAME),
            os.path.join(Config.DATA_DIR, Config.PRM_FILENAME)
        )

        # 複数相の読み込みループ
        self.phases = []
        for i, cif_file in enumerate(Config.CIF_FILENAMES):
            phase = self.gpx.add_phase(
                os.path.join(Config.DATA_DIR, cif_file),
                phasename=f'Y2O3_Phase_{i}' if len(Config.CIF_FILENAMES) > 1 else 'Y2O3',
                histograms=[self.hist1]
            )
            # 全原子を等方性温度因子(Isotropic)に変換
            for atom in phase.data['Atoms']:
                atom[9] = 'I' 
            self.phases.append(phase)

    def refine(self, param_dict):
        """パラメータを指定して精密化を実行し、Rwpを返す"""
        try:
            self.gpx.do_refinements([param_dict])
            return self.gpx.histograms().get_wR()
        except:
            return None # 計算エラー

# 目的関数: Optunaが試行錯誤する内容
def objective(trial):
    # ---------------------------------------------
    # 1. パラメータ決定フェーズ (設定 or 自動探索)
    # ---------------------------------------------
    
    # [解析範囲]
    if Config.MANUAL_LIMITS:
        limits = Config.MANUAL_LIMITS
    else:
        # 範囲を少し揺らして探索させる
        lb = trial.suggest_float('Limits_LB', 10.0, 20.0)
        ub = trial.suggest_float('Limits_UB', 80.0, 120.0)
        limits = [lb, ub]

    # [バックグラウンド]
    if Config.MANUAL_BG_FUNC:
        bg_func = Config.MANUAL_BG_FUNC
    else:
        bg_func = trial.suggest_categorical('BG_Func', 
            ['chebyschev', 'cosine', 'Q^2 power series', 'inv interpolate'])
            
    if Config.MANUAL_BG_DEGREE:
        bg_deg = Config.MANUAL_BG_DEGREE
    else:
        bg_deg = trial.suggest_int('BG_Deg', 3, 15)

    # [精密化フラグ]
    # Zero
    if Config.FORCE_REFINE_ZERO is not None:
        refine_zero = Config.FORCE_REFINE_ZERO
    else:
        refine_zero = trial.suggest_categorical('Refine_Zero', [True, False])

    # Sample Displacement (Scaleは常にON推奨)
    refine_sample = []
    if Config.FORCE_REFINE_SAMPLE is not None:
        if Config.FORCE_REFINE_SAMPLE: refine_sample = ['DisplaceX']
    else:
        if trial.suggest_categorical('Refine_DisplaceX', [True, False]):
            refine_sample.append('DisplaceX')
    refine_sample.append('Scale') # 必須

    # Peak Shape (U, V, W)
    refine_peak = []
    peak_prms = ['U', 'V', 'W']
    for p in peak_prms:
        if Config.FORCE_REFINE_PEAK is not None:
            if Config.FORCE_REFINE_PEAK: refine_peak.append(p)
        else:
            if trial.suggest_categorical(f'Refine_Peak_{p}', [True, False]):
                refine_peak.append(p)

    # ---------------------------------------------
    # 2. 精密化実行フェーズ (Sequential Refinement)
    # ---------------------------------------------
    try:
        project = ProjectBBO(trial.number)
        
        # Step 0: 範囲とバックグラウンド (土台)
        step0 = {
            'set': {
                'Limits': limits,
                'Background': {'type': bg_func, 'no. coeffs': bg_deg, 'refine': True}
            }
        }
        project.refine(step0)

        # Step 1: ゼロ点・試料位置・スケール (軸)
        step1 = {'set': {
            'Instrument Parameters': ['Zero'] if refine_zero else [],
            'Sample Parameters': refine_sample
        }}
        project.refine(step1)

        # Step 2: ピーク形状 (シルエット)
        if refine_peak:
            step2 = {'set': {'Instrument Parameters': refine_peak}}
            project.refine(step2)

        # Step 3: 格子定数 (ピーク位置)
        # ※ 失敗しても止まらないようにtryで囲む
        try:
            step3 = {'set': {'Cell': True}}
            project.refine(step3)
        except:
            pass

        # Step 4: 原子座標・温度因子 (中身) -> 最終スコア
        step4 = {'set': {'Atoms': {'all': 'XU'}}}
        final_rwp = project.refine(step4)

        if final_rwp is None: return Config.ERROR_PENALTY

        # ---------------------------------------------
        # 3. 制約チェックフェーズ (物理的妥当性)
        # ---------------------------------------------
        # 全相の温度因子が正であるかチェック
        for phase in project.gpx.phases():
            u_iso_vals = [atom.uiso for atom in phase.atoms()]
            if u_iso_vals and min(u_iso_vals) < 0:
                return Config.ERROR_PENALTY

        # Rwpが閾値を超えていたら失敗扱い
        if final_rwp > Config.MAX_RWP_THRESHOLD:
            return Config.ERROR_PENALTY

        return final_rwp

    except Exception as e:
        # 予期せぬエラー
        return Config.ERROR_PENALTY

In [ ]:
# 最適化の実行
study = optuna.create_study(
    study_name=f"{Config.STUDY_NAME}_seed{Config.RANDOM_SEED}",
    storage=f"sqlite:///{Config.WORK_DIR}/history.db",
    load_if_exists=True,
    sampler=optuna.samplers.TPESampler(seed=Config.RANDOM_SEED),
    direction='minimize'
)

print(f"--- 解析開始: {Config.N_TRIALS} 回試行 (Jobs: {Config.N_JOBS}) ---")
study.optimize(objective, n_trials=Config.N_TRIALS, n_jobs=Config.N_JOBS)

print("\n=== 解析完了 ===")
print(f"Best Rwp: {study.best_value:.3f}%")
print("Best Params:", study.best_params)

In [ ]:
# 結果の可視化と診断
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import GSASIIscriptable as G2sc

def plot_best_result():
    # 最良のGPXファイルを読み込み
    best_gpx = os.path.join(Config.WORK_DIR, f'{Config.STUDY_NAME}_seed{Config.RANDOM_SEED}_trial_{study.best_trial.number}.gpx')
    gpx = G2sc.G2Project(best_gpx)
    hist = gpx.histogram(0)  # 最初のヒストグラムを取得
    
    # データの取得
    x = hist.getdata("X")
    y_obs = hist.getdata("Yobs")
    y_calc = hist.getdata("Ycalc")
    resid = hist.getdata("Residual")
    rwp = hist.get_wR()
    
    # 簡易GOF計算 (GSASIIのバージョンにより取得法が異なるため近似)
    # GOF = sqrt(chi^2)
    try:
        gof = hist.data.get('GOF', 0.0)
    except:
        gof = 0.0

    # プロット
    fig = plt.figure(figsize=(12, 8))
    gs = GridSpec(5, 1, figure=fig)
    ax1 = fig.add_subplot(gs[:4, :])
    ax2 = fig.add_subplot(gs[4, :])
    
    ax1.set_title(f"Y2O3 - Best Fit: Rwp={rwp:.2f}%, GOF={gof:.2f}")
    ax1.plot(x, y_obs, 'k+', label='Observed', alpha=0.6)
    ax1.plot(x, y_calc, 'r-', label='Calculated', linewidth=1.5)
    ax1.legend()
    ax1.set_ylabel('Intensity')
    ax1.set_xticklabels([])
    
    ax2.plot(x, resid, 'b-', linewidth=1)
    ax2.set_ylabel('Residual')
    ax2.set_xlabel('2-theta (deg)')
    
    plt.tight_layout()
    plt.show()

    # 自動コメント
    print("\n【診断】")
    if rwp < 15.0 and 1.0 <= gof <= 2.5:
        print("✅ 良好な解析結果です。原子位置や格子定数を確認してください。")
    elif rwp > 30.0:
        print("⚠️ Rwpが高すぎます。Configで解析範囲(MANUAL_LIMITS)を狭めるか、初期構造モデル(CIF)が正しいか確認してください。")
    elif gof < 1.0:
        print("⚠️ GOFが低すぎます。過学習またはデータのエラー値の問題の可能性があります。")
    else:
        print("ℹ️ まずまずの結果です。必要に応じて試行回数(N_TRIALS)を増やしてみてください。")
    
    # Y2O3の格子定数を出力
    print("\n【Y2O3 結晶構造パラメータ】")
    for phase in gpx.phases():
        print(f"Phase: {phase.name}")
        cell = phase.data['General']['Cell'][1:7]
        print(f"  結晶系: 立方晶系 (Cubic)")
        print(f"  空間群: Ia-3 (No. 206)")
        print(f"  格子定数: a={cell[0]:.4f} Å")
        print(f"  格子角: α={cell[3]:.2f}°, β={cell[4]:.2f}°, γ={cell[5]:.2f}°")
        print(f"  原子数: {len(phase.atoms())}")
        print("\n  原子座標:")
        for atom in phase.atoms():
            print(f"    {atom.element:2s}: x={atom.x:.4f}, y={atom.y:.4f}, z={atom.z:.4f}, Uiso={atom.uiso:.4f}")

plot_best_result()

## 最適化履歴の可視化

Optunaの最適化過程を確認します。

In [ ]:
# 最適化履歴のプロット
import optuna.visualization as vis

# 最適化の収束過程
fig1 = vis.plot_optimization_history(study)
fig1.show()

# パラメータの重要度
try:
    fig2 = vis.plot_param_importances(study)
    fig2.show()
except:
    print("パラメータ重要度の計算には十分な試行回数が必要です。")

# パラメータ間の相関
try:
    fig3 = vis.plot_parallel_coordinate(study)
    fig3.show()
except:
    print("パラレル座標プロットの作成には十分な試行回数が必要です。")